# LangGraph MCP (Model Context Protocol) 완전 정복 튜토리얼

## 개요

**MCP(Model Context Protocol)** 는 **AI 에이전트가 외부 도구와 소통하는 표준화된 방법** 을 제공하는 혁신적인 프로토콜입니다.

### MCP의 핵심 특징

기존 AI 시스템에서는 각각의 도구나 서비스마다 다른 연결 방식을 사용해야 했습니다. MCP는 이러한 문제를 해결합니다:

| 특징 | 설명 | 장점 |
|------|------|------|
| **표준화된 인터페이스** | 모든 도구가 동일한 방식으로 연결 | 개발 효율성 향상, 학습 비용 감소 |
| **동적 도구 발견** | 런타임에 사용 가능한 도구를 자동으로 찾아서 활용 | 유연한 시스템 구성 |
| **다양한 전송 방식** | stdio, HTTP, WebSocket 등 환경에 맞는 연결 방법 지원 | 다양한 환경 호환성 |
| **확장 가능성** | 새로운 도구를 쉽게 추가하고 관리 | 시스템 성장성 |

### 학습 목표

이 튜토리얼을 통해 다음 내용을 배우게 됩니다:

1. **MCP 기본 개념** - MCP의 정의와 필요성 이해
2. **MCP 서버 구축** - 실제 MCP 서버 구현 방법
3. **MultiServerMCPClient 활용** - 여러 서버를 효율적으로 관리하는 방법
4. **React Agent 통합** - LangGraph의 React Agent와 MCP 결합 방법
5. **ToolNode 활용** - 세밀한 워크플로우 제어 기법
6. **외부 MCP 서버 활용** - 3rd party MCP 서버 연동 방법
7. **실전 예제** - 복합 AI 에이전트 시스템 구축

### MCP 개념 이해를 위한 비유

**MCP** 를 **기업의 IT 인프라** 에 비유하면 다음과 같습니다:

```
본사 (AI 에이전트)
├── 표준 통신 규약 (MCP 프로토콜)
├── 다양한 부서들 (MCP 서버들)
│   ├── 인사부 (사용자 관리 도구)
│   ├── 회계부 (데이터 처리 도구)
│   └── IT부 (시스템 관리 도구)
└── 통합 관리 시스템 (MultiServerMCPClient)
```

### 핵심 가치

MCP의 핵심 가치는 **"하나의 표준으로 모든 도구와 연결하기"** 입니다. 이를 통해 AI 시스템은 무한한 확장성을 갖게 됩니다.

### 전제 조건

이 튜토리얼을 진행하기 전에 다음 사항을 확인하시기 바랍니다:

- Python 기본 문법 이해
- LangChain/LangGraph 기본 개념 숙지
- 비동기 프로그래밍(async/await) 기본 이해

---

## Part 1: MCP 핵심 개념 이해

### MCP(Model Context Protocol) 정의

**MCP** 는 **AI 애플리케이션과 외부 도구 간의 소통을 표준화** 한 오픈 프로토콜입니다. 

### 기존 방식과의 비교

현대 기업에서 다양한 부서가 협업하는 상황을 통해 MCP의 필요성을 이해할 수 있습니다:

#### 기존 방식의 문제점

```
영업부 ←→ 개발팀: 이메일로 소통
마케팅 ←→ 디자인: 슬랙으로 소통  
기획팀 ←→ 운영팀: 회의실에서 소통
```

**결과**: 각기 다른 소통 방식으로 인한 혼란과 비효율성

#### MCP 적용 후

```
통합 업무 플랫폼 (MCP)
├── 모든 부서가 동일한 인터페이스 사용
├── 실시간 정보 공유 및 업데이트
└── 새로운 부서 추가도 쉽게 연결
```

**결과**: 효율적이고 확장 가능한 협업 시스템

### MCP의 4가지 핵심 특징

| 특징 | 설명 | 장점 | 예시 |
|------|------|------|------|
| **표준화된 도구 인터페이스** | 모든 외부 도구가 동일한 방식으로 연결 | 새로운 도구 추가 시 별도 학습 불필요 | 날씨 API, 데이터베이스 모두 동일한 방식으로 호출 |
| **다양한 전송 메커니즘** | stdio, HTTP, WebSocket 등 지원 | 환경에 맞는 최적의 연결 방법 선택 | 로컬은 stdio, 원격은 HTTP |
| **동적 도구 검색** | 런타임에 사용 가능한 도구를 자동 발견 | 도구 추가/제거 시 코드 수정 불필요 | 스마트폰 앱 자동 목록 업데이트와 유사 |
| **확장 가능한 아키텍처** | 멀티 서버, 모듈화 지원 | 시스템 성장에 따른 유연한 확장 | 필요한 기능만 선택적 추가 |

### 전송 메커니즘 상세 비교

| 방식 | 특징 | 사용 시나리오 | 장점 | 단점 |
|------|------|---------------|------|------|
| **stdio** | 로컬 프로세스 간 통신 | 같은 머신 내 서비스 연결 | 빠르고 안정적 | 원격 접속 불가 |
| **HTTP** | 웹 서비스 연결 | 원격 서버, API 연동 | 범용성 높음, 방화벽 통과 | 네트워크 지연 존재 |
| **WebSocket** | 실시간 양방향 통신 | 채팅, 실시간 데이터 스트림 | 실시간성, 양방향 통신 | 연결 관리 복잡 |

### 기존 방식 vs MCP 방식 코드 비교

#### 기존 방식의 한계

```python
# 각 서비스마다 다른 연결 방법 필요
weather_api = WeatherAPI(key="abc123")
database = PostgreSQL(host="localhost")  
file_system = FileManager("/home/user")

# 각각 다른 호출 방식
weather_data = weather_api.get_current_weather("Seoul")
db_result = database.query("SELECT * FROM users") 
files = file_system.list_files()
```

#### MCP 방식의 혁신

```python
# 하나의 클라이언트로 모든 도구 관리
mcp_client = MultiServerMCPClient(server_configs)
tools = await mcp_client.get_tools()

# 통일된 방식으로 모든 도구 사용
for tool in tools:
    result = await tool.invoke({"query": "원하는 작업"})
```

### 필요 패키지

MCP를 사용하기 위해 다음 패키지들이 필요합니다:

| 패키지 | 역할 | 설명 |
|--------|------|------|
| `langchain-mcp-adapters` | MCP 클라이언트 | MCP 서버와의 통신 담당 |
| `langgraph` | 워크플로우 엔진 | 복잡한 AI 워크플로우 구성 |
| `langchain-openai` | LLM 연동 | OpenAI 모델과의 통합 |

```bash
# 패키지 설치
pip install langchain-mcp-adapters langgraph langchain-openai
```

이제 실제 코드를 통해 MCP의 강력함을 체험해보겠습니다.

In [ ]:
# 비동기 처리를 위한 nest_asyncio (Jupyter 환경에서 필수)
import nest_asyncio
from typing import List, Dict, Any

# OpenAI LLM 모델 사용을 위한 import
from langchain_openai import ChatOpenAI

# LangGraph의 사전 구축된 React 에이전트 import
from langgraph.prebuilt import create_react_agent

# 메모리 기반 체크포인터 (대화 상태 저장)
from langgraph.checkpoint.memory import InMemorySaver

# MCP 클라이언트 - 여러 MCP 서버를 동시에 관리하는 핵심 컴포넌트
from langchain_mcp_adapters.client import MultiServerMCPClient

In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
import os

# API KEY 정보로드
load_dotenv(override=True)

# 비동기 호출 활성화
nest_asyncio.apply()

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LangChain-Tutorial")

## Part 2: MCP 서버 아키텍처 이해

**MCP 서버** 는 **특정 기능을 제공하는 독립적인 프로세스** 입니다. 기업 조직에서 각 부서가 전문 분야를 담당하는 것과 같은 개념입니다.

### 서버 분리 개념 이해

```
본사 (Main Application)
├── 데이터분석팀 (mcp_server_rag.py) - RAG 검색 전담
├── 기상정보팀 (mcp_server_local.py) - 날씨 정보 제공  
└── 시간관리팀 (mcp_server_remote.py) - 시간 관련 서비스
```

### 서버 파일 구성 및 특징

이 튜토리얼에서는 세 가지 타입의 MCP 서버를 사용합니다:

| 서버 유형 | 파일명 | 전송방식 | 주요 기능 | 사용 시나리오 |
|-----------|---------|----------|-----------|---------------|
| **로컬 서버** | `mcp_server_local.py` | stdio | 날씨 조회, 파일 관리 | 빠른 응답이 필요한 기본 기능 |
| **원격 서버** | `mcp_server_remote.py` | HTTP/WebSocket | 현재 시간 조회, API 연동 | 네트워크를 통한 원격 서비스 |
| **검색 서버** | `mcp_server_rag.py` | stdio | RAG 문서 검색, 벡터 유사도 검색 | 대용량 문서에서 정확한 정보 검색 |

### 서버별 상세 특징

#### 로컬 서버 (stdio)
- **장점**: 빠른 응답속도, 안정적 연결, 네트워크 지연 없음
- **단점**: 같은 머신에서만 실행 가능
- **적용 분야**: 파일 시스템 접근, 로컬 데이터 처리

#### 원격 서버 (HTTP)
- **장점**: 네트워크를 통한 원격 접속 가능, 확장성
- **단점**: 네트워크 지연 존재, 연결 관리 복잡
- **적용 분야**: 웹 API 연동, 클라우드 서비스 연결

#### 검색 서버 (stdio + RAG)
- **장점**: 대용량 문서 처리, 의미적 검색, 정확한 컨텍스트 제공
- **단점**: 초기 설정 복잡, 리소스 사용량 높음
- **적용 분야**: 문서 검색, 지식베이스 구축

### 서버 분리의 장점

| 장점 | 설명 | 실제 적용 예시 |
|------|------|----------------|
| **모듈성** | 각 기능을 독립적으로 개발하고 관리 | 날씨 서비스에 문제가 생겨도 시간 서비스는 정상 작동 |
| **확장성** | 필요한 기능만 선택적으로 추가 | 새로운 번역 서비스 추가 시 기존 서비스에 영향 없음 |
| **안정성** | 하나의 서버 문제가 다른 서버에 영향을 주지 않음 | 검색 서버 다운 시에도 시간/날씨 조회는 계속 가능 |
| **성능** | 각 서버를 개별적으로 최적화 가능 | 검색 서버는 GPU 사용, 시간 서버는 경량 CPU 사용 |

### MCP 서버 개발 가이드라인

#### 단일 책임 원칙
- 하나의 서버는 하나의 명확한 책임을 가져야 함
- 예: 날씨 서버는 오직 날씨 관련 기능만 제공

#### 상태 독립성
- 서버 간에 상태를 공유하지 않음
- 각 서버는 독립적으로 실행 가능해야 함

#### 표준 인터페이스 준수
- MCP 프로토콜 표준을 정확히 구현
- 도구명, 파라미터, 응답 형식의 일관성 유지

다음 단계에서는 이 서버들을 **MultiServerMCPClient** 로 연결하여 통합 관리하는 방법을 살펴보겠습니다.

## Part 3: MultiServerMCPClient - 통합 관제센터

**MultiServerMCPClient** 는 **여러 MCP 서버를 하나의 인터페이스로 관리** 하는 핵심 컴포넌트입니다.

### IT 관제센터 개념으로 이해하기

대기업의 IT 관제센터를 상상해보세요:

```
IT 관제센터 (MultiServerMCPClient)
├── 모니터링 화면 - 모든 서버 상태 한눈에 파악
├── 통합 제어판 - 하나의 인터페이스로 모든 서버 제어
├── 자동 로드밸런싱 - 부하 분산 자동 처리
└── 에러 복구 시스템 - 장애 발생 시 자동 복구
```

### MultiServerMCPClient의 핵심 기능

| 기능 | 설명 | 코드 예시 | 장점 |
|------|------|-----------|------|
| **서버 구성 관리** | 서버 설정을 딕셔너리로 정의 및 관리 | `server_configs = {"weather": {...}}` | 설정 변경 용이, 버전 관리 가능 |
| **동적 도구 로딩** | 각 서버에서 제공하는 도구들을 자동 수집 | `tools = await client.get_tools()` | 중복 제거, 실시간 업데이트 |
| **통합 인터페이스** | 모든 서버의 도구를 하나의 리스트로 통합 | `for tool in tools: result = await tool.invoke()` | 일관된 사용 방식 |
| **에러 처리 및 복구** | 개별 서버 장애 시 다른 서버는 정상 작동 유지 | 자동 재연결, 에러 로깅 | 시스템 안정성 향상 |

### 서버 구성 설정 상세

#### 기본 구성 형태
```python
server_configs = {
    "서버명": {
        "command": "실행명령어",     # uv, python, node 등
        "args": ["인자1", "인자2"],  # 실행 시 전달할 인자들
        "transport": "전송방식",     # stdio, http, websocket
    }
}
```

#### stdio 전송방식 설정
```python
"local_server": {
    "command": "uv",
    "args": ["run", "python", "server/my_server.py"],
    "transport": "stdio",
}
```

#### HTTP 전송방식 설정
```python
"remote_server": {
    "url": "http://127.0.0.1:8002/mcp",
    "transport": "streamable_http",
}
```

### 개별 서버 관리 vs MultiServerMCPClient 비교

#### 개별 서버 직접 관리 시 문제점

```python
# 각 서버마다 별도 연결 및 관리 필요
weather_client = WeatherMCPClient()
time_client = TimeMCPClient() 
rag_client = RAGMCPClient()

# 서버마다 다른 연결 관리 로직 필요
await weather_client.connect()
await time_client.connect()
await rag_client.connect()

# 에러 처리도 각각 별도로 구현해야 함
try:
    weather_result = await weather_client.get_weather("Seoul")
except WeatherClientError:
    # 날씨 서버 전용 에러 처리
    pass
```

#### MultiServerMCPClient 사용 시 장점

```python
# 하나의 클라이언트로 모든 서버 관리
client = MultiServerMCPClient(all_server_configs)
tools = await client.get_tools()  # 모든 도구 한번에 로드

# 통일된 방식으로 모든 도구 사용
for tool in tools:
    try:
        result = await tool.invoke({"query": "작업 내용"})
    except Exception as e:
        # 통일된 에러 처리
        print(f"도구 {tool.name} 실행 중 오류: {e}")
```

### 실제 사용 패턴

#### 1단계: 설정 정의
```python
server_configs = {
    "weather": {"command": "uv", "args": [...], "transport": "stdio"},
    "time": {"url": "http://...", "transport": "streamable_http"},
}
```

#### 2단계: 클라이언트 생성 및 도구 로드
```python
client = MultiServerMCPClient(server_configs)
tools = await client.get_tools()
```

#### 3단계: Agent와 연결
```python
agent = create_react_agent(llm, tools, checkpointer=InMemorySaver())
```

### 고급 기능

#### 도구 필터링
특정 조건에 맞는 도구만 선택적으로 사용할 수 있습니다:
```python
# 날씨 관련 도구만 필터링
weather_tools = [tool for tool in tools if "weather" in tool.name.lower()]
```

#### 서버 상태 모니터링
```python
# 연결된 서버 목록 확인
print(f"활성 서버 수: {len(client.servers)}")
for server_name in client.servers:
    print(f"서버 {server_name}: 연결됨")
```

MultiServerMCPClient를 통해 복잡한 다중 서버 환경을 단순하고 일관성 있게 관리할 수 있습니다.

In [ ]:
# MultiServerMCPClient 설정 예제
async def setup_mcp_client(server_configs: List[Dict[str, Any]]):
    """
    MCP 클라이언트를 설정하고 도구를 가져옵니다.

    Args:
        server_configs: 서버 설정 정보 리스트

    Returns:
        tuple: (MCP 클라이언트, 도구 리스트)
    """

    # MultiServerMCPClient 인스턴스 생성 - 여러 MCP 서버를 통합 관리
    client = MultiServerMCPClient(server_configs)

    # 모든 연결된 서버로부터 사용 가능한 도구들을 수집
    tools = await client.get_tools()

    # 로드된 도구 정보를 콘솔에 출력 (디버깅 및 확인용)
    print(f"✅ {len(tools)} 개의 MCP 도구가 로드되었습니다:")
    for tool in tools:
        print(f"  - {tool.name}")  # 각 도구의 이름 출력

    return client, tools

In [ ]:
# 서버 구성 정의 - 로컬 날씨 서버 설정
server_configs = {
    "weather": {
        "command": "uv",  # uv 패키지 매니저를 사용하여 실행
        "args": [
            "run",
            "python",
            "server/mcp_server_local.py",
        ],  # 날씨 서버 스크립트 실행
        "transport": "stdio",  # 표준 입출력 방식으로 통신
    },
}

# MCP 클라이언트 생성 및 도구 로딩
client, tools = await setup_mcp_client(server_configs=server_configs)

In [ ]:
# OpenRouter를 통한 GPT 모델 사용 설정
llm = ChatOpenAI(
    model="openai/gpt-4.1",
    temperature=0,
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url=os.getenv("OPENROUTER_BASE_URL"),
)  # 일관된 결과를 위해 temperature=0

# LangGraph의 사전 구축된 React Agent 생성
# - llm: 사용할 언어 모델
# - tools: MCP에서 로드한 도구들
# - checkpointer: 대화 상태를 메모리에 저장하여 연속 대화 지원
agent = create_react_agent(
    llm, tools, checkpointer=InMemorySaver()  # 상태 저장을 위한 체크포인터
)

In [ ]:
# 스트리밍 출력과 랜덤 UUID 생성을 위한 유틸리티 함수 import
from langchain_teddynote.messages import astream_graph, random_uuid
from langchain_core.runnables import RunnableConfig

# 대화 스레드를 구분하기 위한 고유 ID 생성
config = RunnableConfig(configurable={"thread_id": random_uuid()})

# 에이전트 실행 - 서울 날씨 조회 요청
# astream_graph: 그래프 실행 결과를 스트리밍으로 출력 (실시간으로 처리 과정 확인 가능)
response = await astream_graph(
    agent,  # 실행할 에이전트
    inputs={
        "messages": [("human", "안녕하세요. 서울의 날씨를 알려주세요.")]
    },  # 사용자 입력 메시지
    config=config,  # 대화 설정 (스레드 ID 포함)
)

### HTTP 전송 방식 - 원격 서버 연결

**HTTP 전송 방식** 은 네트워크를 통해 원격 MCP 서버와 연결하는 방법입니다. 인터넷을 통한 API 호출과 동일한 방식입니다.

### 원격 연결 개념 이해

```
Local Client
↕️ 인터넷 연결
Remote MCP Server
```

### HTTP vs stdio 비교

| 구분 | **stdio** | **HTTP** |
|------|-----------|----------|
| **위치** | 같은 컴퓨터 내부 | 네트워크로 연결된 원격지 |
| **속도** | 매우 빠름 | 네트워크 지연 존재 |
| **보안** | 내부 통신 | HTTPS로 암호화 가능 |
| **확장성** | 제한적 | 무제한 확장 |
| **설정 복잡도** | 간단 | 네트워크 설정 필요 |
| **장애 대응** | 프로세스 재시작 | 네트워크 상태 의존적 |

### 원격 서버 실행 방법

원격 서버를 사용하려면 먼저 서버를 구동해야 합니다:

```bash
# 터미널에서 다음 명령어 실행
uv run python server/mcp_server_remote.py
```

실행 후 확인사항:
- HTTP 서버가 `http://127.0.0.1:8002/mcp` 주소로 실행
- MCP 프로토콜을 HTTP로 제공 시작
- 네트워크를 통한 원격 접속 준비 완료

### HTTP 방식 사용 시나리오

| 시나리오 | 설명 | 예시 |
|----------|------|------|
| **분산 시스템** | 여러 서버에 기능을 분산 배치 | 검색 서버는 A 머신, 번역 서버는 B 머신 |
| **클라우드 서비스** | AWS, GCP 등의 클라우드 연결 | AWS Lambda 함수를 MCP 서버로 활용 |
| **마이크로서비스** | 서비스별로 독립적인 서버 운영 | 각 팀이 담당 서비스를 독립적으로 관리 |
| **팀 협업** | 팀원들과 공유 서버 사용 | 공통 데이터베이스 접근 서버 |

### HTTP MCP 서버 설정

#### 클라이언트 측 설정
```python
server_config = {
    "server_name": {
        "url": "http://서버주소:포트/엔드포인트",
        "transport": "streamable_http",
    }
}
```

#### 실제 설정 예시
```python
http_server_config = {
    "current_time": {
        "url": "http://127.0.0.1:8002/mcp",
        "transport": "streamable_http",
    },
}
```

### 네트워크 고려사항

#### 보안
- HTTPS 사용 권장 (프로덕션 환경)
- API 키나 인증 토큰 활용
- 방화벽 설정 확인

#### 성능
- 네트워크 지연시간 고려
- 연결 풀링을 통한 성능 최적화
- 타임아웃 설정 적절히 조정

#### 가용성
- 서버 다운타임 대비 재시도 로직
- 로드밸런싱을 통한 고가용성 확보
- 헬스체크를 통한 서버 상태 모니터링

이제 실제 HTTP MCP 서버에 연결하여 원격 통신의 장점을 체험해보겠습니다.

In [ ]:
# HTTP 기반 MCP 서버 설정 예제
http_server_config = {
    "current_time": {
        "url": "http://127.0.0.1:8002/mcp",  # 원격 MCP 서버의 HTTP 엔드포인트
        "transport": "streamable_http",  # HTTP 스트리밍 방식으로 통신
    },
}

# HTTP MCP 서버에 연결하여 클라이언트 생성
client, http_tools = await setup_mcp_client(server_configs=http_server_config)

In [ ]:
# HTTP 서버용 LLM 설정 (OpenRouter 사용)
llm = ChatOpenAI(
    model="openai/gpt-4.1",
    temperature=0,
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url=os.getenv("OPENROUTER_BASE_URL"),
)  # 일관성을 위해 동일한 모델 사용

# HTTP 도구를 사용하는 React Agent 생성
agent = create_react_agent(
    llm,
    http_tools,
    checkpointer=InMemorySaver(),  # HTTP로 로드된 도구들과 메모리 저장소 사용
)

In [ ]:
# 새로운 스레드로 HTTP 서버와 상호작용
from langchain_teddynote.messages import astream_graph, random_uuid
from langchain_core.runnables import RunnableConfig

# 새로운 대화 세션을 위한 고유 ID 생성
config = RunnableConfig(configurable={"thread_id": random_uuid()})

# HTTP MCP 서버를 통해 현재 시간 조회 요청
response = await astream_graph(
    agent,  # HTTP 도구가 연결된 에이전트
    inputs={
        "messages": [("human", "안녕하세요. 현재 시간을 알려주세요.")]
    },  # 시간 조회 요청
    config=config,  # 대화 세션 설정
)

### MCP Inspector - 개발자를 위한 디버깅 도구

**MCP Inspector** 는 MCP 서버를 **웹 브라우저에서 직접 테스트하고 디버깅** 할 수 있는 강력한 개발 도구입니다.

### 개발 도구로서의 위치

개발자라면 누구나 IDE의 디버거를 사용해본 경험이 있을 것입니다. MCP Inspector는 **MCP 서버 전용 디버거** 라고 할 수 있습니다:

```
MCP Inspector (웹 인터페이스)
├── 서버 상태 실시간 모니터링
├── 도구 직접 테스트 및 실행
├── 요청/응답 로그 상세 확인  
└── 에러 발생 시 디버깅 정보 제공
```

### MCP Inspector 주요 기능

| 기능 | 설명 | 사용법 | 장점 |
|------|------|--------|------|
| **서버 탐색** | 연결된 MCP 서버들을 시각적으로 표시 | 서버 목록에서 클릭하여 선택 | 전체 서버 현황 파악 용이 |
| **도구 테스트** | 각 도구의 기능을 직접 실행해볼 수 있음 | 파라미터 입력 후 실행 버튼 클릭 | 코드 작성 없이 도구 동작 확인 |
| **스키마 확인** | 도구의 입력/출력 스키마를 상세히 표시 | 도구 선택 시 자동 표시 | API 문서 역할, 개발 시 참조 |
| **로그 분석** | 실시간 요청/응답 로그를 확인 | 하단 로그 패널에서 확인 | 문제 발생 시 원인 분석 |

### 실행 방법

터미널에서 다음 명령어를 실행하세요:

```bash
npx @modelcontextprotocol/inspector
```

### 실행 후 동작

1. **자동 브라우저 실행**: 기본 브라우저가 자동으로 열림
2. **서버 연결 인터페이스**: MCP 서버들을 연결할 수 있는 UI 제공
3. **실시간 상태 표시**: 연결된 서버들의 상태를 실시간으로 표시
4. **대화형 테스트**: 각 도구를 직접 클릭하여 테스트 가능

### 활용 시나리오

| 시나리오 | 사용 목적 | 구체적 활용 방법 |
|----------|-----------|------------------|
| **서버 개발 중** | 새로 만든 MCP 서버가 올바르게 작동하는지 확인 | 서버 연결 후 제공되는 도구들이 정상적으로 나열되는지 확인 |
| **문제 해결** | 도구 호출이 실패할 때 원인 분석 | 로그 패널에서 에러 메시지 및 스택 트레이스 확인 |
| **성능 측정** | 서버 응답 시간 및 리소스 사용량 확인 | 반복적인 도구 호출을 통한 성능 벤치마킹 |
| **API 문서화** | 서버가 제공하는 도구들의 스펙 확인 | 각 도구의 입력 파라미터와 출력 형식을 문서로 활용 |

### 디버깅 팁

#### 연결 문제 해결
- 서버가 정상적으로 실행 중인지 확인
- 포트 번호와 엔드포인트 URL이 정확한지 검증
- 네트워크 연결 상태 점검

#### 도구 실행 실패 시
- 입력 파라미터 형식이 스키마에 맞는지 확인
- 서버 측 로그에서 에러 메시지 확인
- 필수 파라미터가 누락되지 않았는지 검토

#### 성능 이슈 분석
- 응답 시간이 긴 도구들을 식별
- 대용량 데이터 처리 시 메모리 사용량 모니터링
- 동시 요청 처리 능력 테스트

### 프로덕션 배포 전 체크리스트

MCP Inspector를 사용하여 프로덕션 배포 전에 다음 사항들을 확인하세요:

- [ ] 모든 도구가 정상적으로 로드되는가?
- [ ] 각 도구의 입력/출력이 예상한 대로 동작하는가?
- [ ] 에러 상황에서 적절한 에러 메시지가 반환되는가?
- [ ] 응답 시간이 허용 범위 내에 있는가?
- [ ] 동시 요청 처리가 안정적인가?

MCP Inspector는 MCP 생태계에서 개발 효율성을 크게 향상시키는 필수 도구입니다.

![mcp_inspector](./assets/mcp-inspector.png)

In [ ]:
# RAG(검색 증강 생성) 서버 설정
http_server_config = {
    "rag": {
        "command": "uv",  # uv 패키지 매니저 사용
        "args": ["run", "python", "server/mcp_server_rag.py"],  # RAG 전용 서버 실행
        "transport": "stdio",  # 로컬 서버이므로 stdio 방식 사용
    },
}

# RAG MCP 서버에 연결하여 검색 도구 로딩
client, rag_tools = await setup_mcp_client(server_configs=http_server_config)

In [ ]:
# RAG 서버용 LLM 설정 (OpenRouter 사용)
llm = ChatOpenAI(
    model="openai/gpt-4.1",
    temperature=0,
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url=os.getenv("OPENROUTER_BASE_URL"),
)  # 정확한 검색 결과를 위해 temperature=0

# RAG 도구를 활용하는 React Agent 생성
rag_agent = create_react_agent(
    llm, rag_tools, checkpointer=InMemorySaver()  # RAG 검색 도구와 메모리 저장소 연결
)

In [ ]:
# RAG 에이전트를 사용한 문서 검색 예제
from langchain_teddynote.messages import astream_graph, random_uuid
from langchain_core.runnables import RunnableConfig

# RAG 전용 대화 세션 생성
config = RunnableConfig(configurable={"thread_id": random_uuid()})

# 삼성 가우스 AI에 대한 정보를 MCP RAG 서버에서 검색
# MCP 서버를 명시적으로 사용하도록 요청하여 정확한 정보 검색 유도
_ = await astream_graph(
    rag_agent,  # RAG 검색 도구가 연결된 에이전트
    inputs={
        "messages": [
            (
                "human",
                "삼성전자가 개발한 생성형 AI 의 이름은? mcp 서버를 사용해서 검색해주세요.",
            )
        ]
    },
    config=config,  # RAG 세션 설정
)

In [ ]:
# 동일한 RAG 세션에서 추가 검색 수행
# 이전 대화 맥락을 유지하면서 구글의 Anthropic 투자에 대한 정보 검색
_ = await astream_graph(
    rag_agent,  # 동일한 RAG 에이전트 사용
    inputs={
        "messages": [
            (
                "human",
                "구글이 Anthropic 에 투자하기로 한 금액을 검색해줘",  # 후속 질문
            )
        ]
    },
    config=config,  # 동일한 대화 세션 유지 (이전 맥락 보존)
)

## Part 4: React Agent와 MCP 완벽 통합

**React Agent** 는 **추론(Reason)** 과 **행동(Act)** 을 반복하는 지능형 에이전트 패턴입니다. MCP 도구와 결합하면 **자율적으로 문제를 해결하는 강력한 AI 시스템** 을 구축할 수 있습니다.

### 인간의 문제 해결 과정 이해

일상에서 우리가 복잡한 문제를 해결하는 과정을 생각해보세요:

```
문제 상황: "내일 부산 출장, 준비해야 할 것은?"

1. 추론(Reason): "날씨 정보가 필요하다"
2. 행동(Act): 날씨 앱을 열어 부산 날씨 확인
3. 추론(Reason): "비가 올 것 같다. 우산이 필요하겠다"
4. 행동(Act): 우산을 가방에 챙김
5. 추론(Reason): "준비 완료!"
```

### ReAct 패턴의 작동 원리

#### 기본 실행 흐름

```
사용자 질문 → Agent 시작
     ↓
[Reason] "무엇이 필요한가?" → 상황 분석 및 계획 수립
     ↓  
[Act] 필요한 도구 실행 → MCP 도구 호출
     ↓
결과 분석 → 목표 달성했나?
     ↓
아직 미완료라면 다시 Reason & Act
     ↓
목표 달성 시 최종 답변 제공
```

#### 구성요소별 역할

| 구성요소 | 역할 | 구체적 예시 | 특징 |
|----------|------|-------------|------|
| **React Agent** | 논리적 추론 및 계획 수립 | "날씨와 시간 정보가 모두 필요하다" | 전략적 사고, 단계별 계획 |
| **MCP Tools** | 구체적인 작업 실행 | `get_weather()`, `get_time()` | 실제 기능 수행, 데이터 수집 |
| **LLM** | 자연어 이해 및 생성 | 사용자 질문 해석, 답변 생성 | 언어적 추론, 맥락 이해 |
| **Memory** | 대화 맥락 유지 | 이전 정보를 기억하며 연속 대화 | 상태 지속성, 컨텍스트 보존 |

### React Agent의 장점

| 장점 | 설명 | 실제 예시 |
|------|------|-----------|
| **자율성** | 복잡한 작업을 단계적으로 분해하여 자동 처리 | "서울 날씨와 뉴욕 시간을 알려줘" → 두 개의 서로 다른 도구를 순차적으로 호출 |
| **유연성** | 상황에 따라 다른 도구를 적절히 선택 | 정보 검색이 필요하면 검색 도구, 계산이 필요하면 계산 도구 사용 |
| **투명성** | 각 단계의 추론 과정을 명확히 볼 수 있음 | "먼저 날씨를 확인하겠습니다" → "날씨 정보를 바탕으로 옷차림을 추천하겠습니다" |
| **확장성** | 새로운 도구 추가 시 자동으로 활용 | 번역 도구를 추가하면 다국어 질문에 자동으로 번역 도구 활용 |

### 단순 도구 호출 vs React Agent 비교

#### 단순 도구 호출
```python
# 개발자가 직접 도구 호출 순서를 정의
weather = await weather_tool.invoke({"city": "Seoul"})
time = await time_tool.invoke({"timezone": "Asia/Seoul"})
result = f"날씨: {weather}, 시간: {time}"
```

**한계점**:
- 미리 정의된 순서대로만 실행
- 동적 상황 대응 불가
- 복잡한 로직 구현 어려움

#### React Agent 방식
```python
# Agent가 상황에 따라 자율적으로 도구 선택 및 실행
agent = create_react_agent(llm, tools, checkpointer=InMemorySaver())
result = await agent.ainvoke({"messages": [("human", "서울 날씨와 현재 시간을 알려줘")]})
```

**장점**:
- 상황에 따른 동적 도구 선택
- 복잡한 다단계 작업 자동 처리
- 예상치 못한 상황에 대한 유연한 대응

### React Agent + MCP 통합의 시너지

#### 문제 해결 시나리오
"오늘 서울 날씨가 좋으면 한강공원 정보를 찾아줘"

1. **Reason**: 먼저 서울 날씨를 확인해야 함
2. **Act**: 날씨 MCP 도구 호출
3. **Reason**: 날씨가 좋음. 이제 한강공원 정보 필요
4. **Act**: 검색 MCP 도구로 한강공원 정보 검색
5. **Reason**: 충분한 정보 수집 완료
6. **최종 답변**: 날씨 정보와 한강공원 정보를 종합하여 제공

#### 에러 처리 시나리오
날씨 서비스가 일시적으로 다운된 경우:

1. **Reason**: 날씨 정보가 필요하다
2. **Act**: 날씨 도구 호출 → 실패
3. **Reason**: 날씨 서비스가 불가능하다. 대안을 찾자
4. **Act**: 웹 검색 도구로 날씨 정보 검색
5. **최종 답변**: 대안 경로로 획득한 정보 제공

### 실제 구현 패턴

#### 기본 설정
```python
# MCP 도구들과 LLM을 연결한 React Agent
llm = ChatOpenAI(model="openai/gpt-4.1", temperature=0)
agent = create_react_agent(llm, mcp_tools, checkpointer=InMemorySaver())
```

#### 실행
```python
# 자연어 질문을 통한 복잡한 작업 요청
response = await agent.ainvoke({
    "messages": [("human", "복잡한 다단계 작업 요청")]
})
```

React Agent와 MCP의 결합은 AI 시스템에 진정한 자율성과 지능을 부여합니다.

In [ ]:
# React Agent와 MCP를 통합하는 함수 정의
async def create_mcp_react_agent(server_configs: dict):
    """
    MCP 도구를 사용하는 React Agent를 생성합니다.

    Args:
        server_configs: MCP 서버 설정 딕셔너리

    Returns:
        LangGraph 에이전트: MCP 도구가 연결된 React Agent
    """

    # MCP 클라이언트 생성 및 모든 서버의 도구를 통합 로딩
    client, tools = await setup_mcp_client(server_configs=server_configs)

    # OpenRouter를 통한 GPT 모델 사용 (일관성 유지)
    llm = ChatOpenAI(
        model="openai/gpt-4.1",
        temperature=0,
        api_key=os.getenv("OPENROUTER_API_KEY"),
        base_url=os.getenv("OPENROUTER_BASE_URL"),
    )

    # React 패턴을 구현하는 에이전트 생성
    # - 추론(Reason): LLM이 상황을 분석하고 다음 행동 계획
    # - 행동(Act): MCP 도구를 호출하여 구체적 작업 실행
    agent = create_react_agent(
        llm, tools, checkpointer=InMemorySaver()  # 상태 저장을 위한 체크포인터
    )

    return agent

In [ ]:
# 다중 MCP 서버 구성 - 날씨와 시간 서비스 통합
server_configs = {
    "weather": {
        "command": "uv",  # 로컬 날씨 서버
        "args": ["run", "python", "server/mcp_server_local.py"],
        "transport": "stdio",  # 빠른 로컬 통신
    },
    "current_time": {
        "url": "http://127.0.0.1:8002/mcp",  # 원격 시간 서버
        "transport": "streamable_http",  # HTTP 스트리밍 통신
    },
}

# 다중 서버를 통합한 MCP React Agent 생성
agent = await create_mcp_react_agent(server_configs)

In [ ]:
# React Agent 통합 테스트 - 다중 서버 활용
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import random_uuid, astream_graph

# 멀티 도구 테스트를 위한 대화 세션 생성
config = RunnableConfig(configurable={"thread_id": random_uuid()})

# 1차 테스트: 시간 정보 요청 (HTTP 서버 사용)
await astream_graph(
    agent, inputs={"messages": [("human", "현재 시간을 알려주세요")]}, config=config
)

# 2차 테스트: 날씨 정보 요청 (로컬 서버 사용)
# 동일한 세션을 사용하여 대화 맥락 유지
await astream_graph(
    agent,
    inputs={"messages": [("human", "현재 서울의 날씨도 알려주세요")]},
    config=config,  # 이전 대화와 연결된 동일한 세션 사용
)

## Part 5: ToolNode로 고급 워크플로우 구축

**ToolNode** 는 **더 세밀한 제어가 가능한 커스텀 워크플로우** 를 만들 때 사용하는 고급 컴포넌트입니다. React Agent가 **자동차** 라면, ToolNode는 **레이싱카** 라고 할 수 있습니다.

### 자동차 vs 레이싱카 비교

#### React Agent (일반 자동차)
```
편리한 자동 운전
├── 사용하기 쉬움 - 목적지만 말하면 알아서 이동
├── 안전함 - 검증된 경로로만 이동
└── 세부 제어 제한 - 운전자가 직접 조작하기 어려움
```

#### ToolNode (레이싱카)
```
전문가용 수동 운전  
├── 최고 성능 - 원하는 정확한 제어 가능
├── 맞춤형 설계 - 복잡한 로직 구현 가능
└── 개발 노력 필요 - 더 많은 설정과 코드 작성 필요
```

### ToolNode의 핵심 장점

| 장점 | 설명 | 실제 활용 예시 | React Agent와의 차이점 |
|------|------|----------------|----------------------|
| **워크플로우 커스터마이징** | 복잡한 조건부 로직 구현 가능 | "A 도구 실행 → 결과가 특정 조건이면 B 도구, 아니면 C 도구" | React Agent는 LLM이 자동 판단 |
| **성능 최적화** | 불필요한 LLM 호출 최소화 | 단순 조건 분기는 코드로 처리, LLM은 필요시만 호출 | React Agent는 매 단계마다 LLM 호출 |
| **고급 패턴 구현** | 조건부 분기, 루프, 병렬 처리 | 여러 도구를 동시에 실행하여 결과를 종합 | React Agent는 순차적 실행 |
| **에러 처리 세밀화** | 도구별 맞춤 에러 처리 전략 | 검색 실패 시 대안 검색, 재시도 횟수 제한 | React Agent는 일반적 에러 처리 |

### 사용 시나리오 비교

#### ToolNode를 사용해야 하는 경우

| 상황 | 이유 | 구체적 예시 |
|------|------|-------------|
| **복잡한 비즈니스 로직** | 다단계 승인이나 복잡한 조건 분기 | "주문금액 > 100만원이면 매니저 승인 → 승인되면 결제 → 실패시 고객 알림" |
| **성능이 중요한 시스템** | 실시간 처리나 대용량 데이터 처리 | 수천 개의 문서를 병렬로 처리하여 결과 집계 |
| **정교한 에러 처리** | 각 단계별로 다른 장애 복구 전략 | "API 호출 실패 → 3회 재시도 → 대안 API 호출 → 캐시된 데이터 사용" |
| **특수한 워크플로우** | 기존 패턴으로 해결 불가한 요구사항 | 조건에 따라 서로 다른 분석 알고리즘을 선택적으로 적용 |

#### React Agent를 사용하는 경우

| 상황 | 이유 | 구체적 예시 |
|------|------|-------------|
| **간단한 작업** | 단순 질문-답변이나 정보 조회 | "오늘 날씨 어때?" "파일 목록 보여줘" |
| **빠른 프로토타이핑** | 개념 증명이나 데모 제작 | 새로운 아이디어를 빠르게 테스트 |
| **일반적인 용도** | 표준적인 AI 어시스턴트 기능 | 대화형 질의응답, 일반적인 정보 검색 |
| **개발 리소스 제한** | 빠른 개발이 우선인 경우 | 시간이나 인력이 제한된 프로젝트 |

### ToolNode 워크플로우 구성 요소

#### StateGraph
```python
workflow = StateGraph(AgentState)
```
- 워크플로우의 전체 구조를 정의
- 각 노드 간의 연결 관계를 명시

#### 노드 정의
```python
# 에이전트 노드: LLM을 활용한 의사결정
async def agent_node(state: AgentState):
    response = await llm_with_tools.ainvoke(state["messages"])
    return {"messages": [response]}

# 도구 노드: 실제 도구 실행
tool_node = ToolNode(tools)
```

#### 조건부 엣지
```python
workflow.add_conditional_edges("agent", tools_condition)
```
- 상황에 따라 다른 경로로 분기
- 복잡한 의사결정 로직 구현

### 실전 예제: 하이브리드 검색 시스템

#### 요구사항
1. **웹 검색**: Tavily로 최신 정보 수집
2. **문서 검색**: MCP RAG로 내부 문서 검색
3. **시간 정보**: MCP 시간 서버로 컨텍스트 추가
4. **통합 분석**: 모든 정보를 종합하여 최종 답변 생성

#### 워크플로우 설계
```python
workflow = StateGraph(AgentState)

# 노드 추가
workflow.add_node("agent", agent_node)  # LLM 의사결정
workflow.add_node("tools", tool_node)   # 도구 실행

# 엣지 연결
workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", tools_condition)
workflow.add_edge("tools", "agent")
workflow.add_edge("agent", END)
```

### ToolNode vs React Agent 성능 비교

| 측면 | React Agent | ToolNode | 상황별 권장 |
|------|------------|----------|------------|
| **개발 속도** | 빠름 | 보통 | 프로토타입: React Agent |
| **실행 성능** | 보통 | 높음 | 프로덕션: ToolNode |
| **유지보수** | 쉬움 | 복잡 | 장기 운영: 상황에 따라 |
| **디버깅** | 어려움 | 쉬움 | 복잡한 로직: ToolNode |

### 마이그레이션 가이드

#### React Agent에서 ToolNode로 전환 시점
1. **성능 병목** 이 발생할 때
2. **복잡한 비즈니스 로직** 이 필요할 때
3. **세밀한 제어** 가 요구될 때
4. **프로덕션 배포** 를 준비할 때

ToolNode는 고급 사용자를 위한 강력한 도구이지만, 대부분의 경우 React Agent로도 충분합니다. 프로젝트의 요구사항과 팀의 역량을 고려하여 적절한 선택을 하시기 바랍니다.

In [ ]:
# ToolNode 기반 고급 워크플로우 구현
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_teddynote.graphs import visualize_graph
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage
from typing import Annotated, List, Dict, Any
from langchain_openai import ChatOpenAI
from typing import TypedDict
from langchain_tavily import TavilySearch


# 에이전트의 상태를 정의하는 스키마
class AgentState(TypedDict):
    """
    에이전트 상태 - 워크플로우 전반에 걸쳐 유지되는 데이터
    """

    messages: Annotated[List[BaseMessage], add_messages]  # 대화 메시지 (자동 누적)
    context: Dict[str, Any] = None  # 추가 컨텍스트 정보 (검색 결과, 메타데이터 등)


async def create_mcp_workflow(server_configs: dict):
    """
    MCP 도구를 활용하는 고급 커스텀 워크플로우 생성

    Args:
        server_configs: MCP 서버 설정 딕셔너리

    Returns:
        컴파일된 LangGraph 애플리케이션
    """

    # MCP 클라이언트에서 도구들을 로드
    client, tools = await setup_mcp_client(server_configs=server_configs)

    # Tavily 웹 검색 도구를 MCP 도구 목록에 추가 (하이브리드 검색 시스템 구성)
    tool = TavilySearch(max_results=2)  # 최대 2개 검색 결과로 제한
    tools.append(tool)

    # OpenRouter를 통한 GPT 모델과 도구 바인딩
    llm = ChatOpenAI(
        model="openai/gpt-4.1",
        temperature=0,
        api_key=os.getenv("OPENROUTER_API_KEY"),
        base_url=os.getenv("OPENROUTER_BASE_URL"),
    )
    llm_with_tools = llm.bind_tools(tools)  # LLM에 모든 도구를 연결

    # StateGraph로 커스텀 워크플로우 정의
    workflow = StateGraph(AgentState)

    # 에이전트 노드: LLM이 상황을 분석하고 도구 호출을 결정
    async def agent_node(state: AgentState):
        """
        LLM을 호출하여 응답 생성 및 도구 사용 결정

        Args:
            state: 현재 에이전트 상태

        Returns:
            업데이트된 상태 (새 메시지 추가)
        """
        response = await llm_with_tools.ainvoke(state["messages"])
        return {"messages": [response]}

    # ToolNode 생성: 실제 도구 실행을 담당
    tool_node = ToolNode(tools)

    # 워크플로우 그래프 구성
    workflow.add_node("agent", agent_node)  # 추론 및 계획 수립 노드
    workflow.add_node("tools", tool_node)  # 도구 실행 노드

    # 엣지 연결 (워크플로우 흐름 정의)
    workflow.add_edge(START, "agent")  # 시작 -> 에이전트
    workflow.add_conditional_edges(
        "agent", tools_condition
    )  # 조건부: 도구 호출 필요시 tools로, 아니면 END로
    workflow.add_edge("tools", "agent")  # 도구 실행 후 -> 에이전트로 돌아가서 결과 분석
    workflow.add_edge("agent", END)  # 최종 완료시 -> 종료

    # 워크플로우 컴파일 (메모리 저장소 포함)
    app = workflow.compile(checkpointer=InMemorySaver())

    # 워크플로우 구조를 시각적으로 확인
    visualize_graph(app)

    return app

In [ ]:
# ToolNode 워크플로우를 위한 다중 MCP 서버 설정
server_configs = {
    "weather": {
        "command": "uv",  # 로컬 날씨 서버
        "args": ["run", "python", "server/mcp_server_local.py"],
        "transport": "stdio",  # 빠른 로컬 통신
    },
    "current_time": {
        "url": "http://127.0.0.1:8002/mcp",  # 원격 시간 서버
        "transport": "streamable_http",  # HTTP 스트리밍 통신
    },
}

In [ ]:
# 고급 MCP 워크플로우 애플리케이션 생성
# - MCP 도구들과 Tavily 웹 검색을 통합한 하이브리드 시스템
mcp_app = await create_mcp_workflow(server_configs)

In [ ]:
# ToolNode 워크플로우 테스트 - 시간 정보 요청
# 커스텀 워크플로우가 MCP 시간 서버를 올바르게 활용하는지 확인
_ = await astream_graph(
    mcp_app, inputs={"messages": [("human", "현재 시간을 알려주세요")]}, config=config
)

In [ ]:
# 복합 워크플로우 테스트 - 시간 + 웹 검색 통합 작업
# MCP 시간 서버와 Tavily 웹 검색을 연계한 고급 작업 수행
_ = await astream_graph(
    mcp_app,
    inputs={
        "messages": [
            ("human", "오늘 뉴스를 검색해주세요. 검색시 시간을 조회한 뒤 처리하세요.")
        ]
    },
    config=config,  # 동일한 세션으로 연속 대화
)

## Part 6: 외부 MCP 서버 생태계 활용하기

**3rd Party MCP 서버** 를 활용하면 이미 검증된 외부 서비스들을 손쉽게 연결할 수 있습니다. 스마트폰의 앱 스토어에서 필요한 앱을 다운로드하는 것과 같은 개념입니다.

### Smithery AI - MCP 생태계의 허브

**Smithery AI** 는 **MCP 서버들의 중앙 집중형 허브** 로, 다양한 검증된 MCP 서비스들을 제공합니다.

### 외부 서비스 연동 개념

```
우리 회사 (Main Application)
     ↕️ MCP 프로토콜
Smithery AI (외부 서비스 허브)
├── 검색 서비스 (Context7)
├── 데이터 분석 서비스  
├── 소셜 미디어 연동
├── 이메일 자동화
└── 개발 도구 통합
```

### Smithery AI 플랫폼 정보

| 항목 | 설명 | 특징 |
|------|------|------|
| **웹사이트** | https://smithery.ai/ | MCP 서버의 GitHub 마켓플레이스 |
| **제공 서비스** | 커뮤니티에서 개발한 다양한 MCP 서버들 | 검증된 품질, 활발한 커뮤니티 |
| **사용 방식** | NPM 패키지 매니저를 통한 간편 설치 | `npx @smithery/cli` 명령어 사용 |
| **라이선스** | 대부분 오픈소스 | 자유로운 상업적 이용 |

### Context7 MCP 서버 - 지능형 문서 검색

**Context7** 은 Smithery AI에서 제공하는 **고성능 RAG 검색 서비스** 입니다:

#### 핵심 기능

| 기능 | 설명 | 활용 예시 | 장점 |
|------|------|-----------|------|
| **대규모 문서 검색** | LangGraph, LangChain 등 최신 기술 문서 | "LangGraph의 새로운 기능 찾기" | 방대한 문서 커버리지 |
| **실시간 업데이트** | 최신 라이브러리 문서를 실시간 반영 | 최신 릴리스 노트 자동 반영 | 정보의 최신성 보장 |
| **정확한 컨텍스트** | 검색 결과의 정확성과 관련성 최적화 | 질문과 정확히 매치되는 답변 | 검색 품질 향상 |
| **다국어 지원** | 영어, 한국어 등 다양한 언어 문서 지원 | 한국어 질문에 대한 영어 문서 검색 | 언어 장벽 해소 |

#### 사용 시나리오

| 시나리오 | 질문 예시 | Context7의 역할 | 결과 |
|----------|-----------|-----------------|------|
| **최신 문서 검색** | "LangGraph의 새로운 기능은?" | 최신 LangGraph 문서에서 새 기능 검색 | 정확하고 최신의 기능 목록 제공 |
| **코드 예제 찾기** | "React Agent 구현 예제 찾아줘" | 코드 예제가 포함된 문서 검색 | 실행 가능한 코드 예제 제공 |
| **문제 해결** | "특정 에러 메시지 해결 방법은?" | 에러 관련 문서 및 해결책 검색 | 구체적인 해결 방법 안내 |
| **API 레퍼런스** | "특정 함수의 파라미터 설명" | API 문서에서 해당 함수 정보 검색 | 정확한 API 사용법 제공 |

### 외부 서버 연동의 장점

#### 비용 효율성
```
자체 구축 비용 vs 외부 서비스 이용
├── 개발 시간: 수개월 → 수분
├── 인프라 비용: 서버 + DB + 유지보수 → 사용량 기반 과금
├── 전문성: 개발팀 학습 비용 → 즉시 활용 가능
└── 업데이트: 수동 관리 → 자동 업데이트
```

#### 품질 보장
- 전문가가 개발하고 유지보수하는 서비스
- 대규모 사용자 베이스를 통한 검증
- 지속적인 품질 개선 및 버그 수정

#### 확장성
- 필요에 따라 새로운 서비스 추가
- 사용량에 따른 자동 스케일링
- 글로벌 서비스 인프라 활용

### 외부 MCP 서버 설정 방법

#### Context7 설정 예시
```python
server_configs = {
    "context7-mcp": {
        "command": "npx",                    # Node.js 패키지 매니저
        "args": [
            "-y",                            # 자동 설치 승인
            "@smithery/cli@latest",          # Smithery AI CLI 도구
            "run",                           # 실행 명령
            "@upstash/context7-mcp",         # Context7 MCP 서버
            "--key",                         # API 키 플래그
            "your-api-key-here",             # 실제 API 키
        ],
        "transport": "stdio",                # 로컬 프로세스로 실행
    },
}
```

### API 키 관리

#### 환경변수 활용
```python
import os
api_key = os.getenv("CONTEXT7_API_KEY")  # .env 파일에서 로드
```

#### 보안 고려사항
- API 키를 코드에 직접 하드코딩하지 않기
- 환경변수나 설정 파일을 통한 안전한 관리
- 정기적인 API 키 로테이션

### 하이브리드 아키텍처 설계

#### 내부 + 외부 서버 조합
```python
server_configs = {
    # 내부 서버들 (빠른 응답, 보안)
    "weather": {"command": "uv", "transport": "stdio"},
    "database": {"command": "python", "transport": "stdio"},
    
    # 외부 서버들 (전문 서비스, 확장성)
    "search": {"command": "npx", "args": [...], "transport": "stdio"},
    "translation": {"url": "https://...", "transport": "http"},
}
```

#### 장애 대응 전략
- 외부 서비스 다운 시 내부 서비스로 폴백
- 여러 외부 서비스를 통한 중복성 확보
- 캐싱을 통한 서비스 의존성 최소화

### 실제 통합 예제

이제 **Context7 MCP 서버** 를 연결하여:
1. **LangGraph 최신 문서 검색**
2. **검색 결과 기반 React Agent 자동 생성**  
3. **내부 MCP + 외부 MCP + Tavily 웹검색 통합 활용**

외부 생태계의 강력함을 직접 체험해보겠습니다.

In [ ]:
# 외부 MCP 서버와 내부 서버 통합 구성
server_configs = {
    # 내부 서버들
    "weather": {
        "command": "uv",  # 로컬 날씨 서버
        "args": ["run", "python", "server/mcp_server_local.py"],
        "transport": "stdio",
    },
    "current_time": {
        "url": "http://127.0.0.1:8002/mcp",  # 원격 시간 서버
        "transport": "streamable_http",
    },
    # 외부 3rd Party 서버 (Smithery AI의 Context7)
    "context7-mcp": {
        "command": "npx",  # Node.js 패키지 매니저를 통해 실행
        "args": [
            "-y",  # 자동 설치 승인
            "@smithery/cli@latest",  # Smithery AI CLI 도구
            "run",
            "@upstash/context7-mcp",  # Context7 MCP 서버
            "--key",
            "smithery-api-key",  # Context7 서비스 API 키
        ],
        "transport": "stdio",  # 로컬 프로세스로 실행
    },
}

# 내부 + 외부 MCP 서버를 통합한 하이브리드 워크플로우 생성
mcp_app = await create_mcp_workflow(server_configs)

In [ ]:
# 하이브리드 MCP 시스템 종합 테스트
# 내부 MCP + 외부 Context7 + Tavily 웹 검색을 모두 활용한 복합 작업
await astream_graph(
    mcp_app,
    inputs={
        "messages": [
            (
                "human",
                "최신 LangGraph 도큐먼트에서 ReAct Agent 관련 내용을 검색하세요. "
                "그런 다음 Tavily 검색을 수행하는 ReAct Agent를 생성하세요. "
                "사용 LLM=gpt-4.1",
            )
        ]
    },
    config=config,  # 연속 대화 세션 유지
)